# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject as ip

# Question 1

**Explain how you solve the model**

In [11]:
# code for solving the model (remember documentation and comments)
#Det her kode var der inden – jeg ved ikke helt om det skal bruges eller ej
#a = np.array([1,2,3])
#b = ip.square(a)
#print(b)

Hm = np.arange(0,24.5,0.5)
Hf = np.arange(0,24.5,0.5)
Lm = np.arange(0,24.5,0.5)
Lf = np.arange(0,24.5,0.5)

print(Hm)
Hm.dtype


[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5 16.  16.5 17.  17.5 18.  18.5 19.  19.5 20.  20.5
 21.  21.5 22.  22.5 23.  23.5 24. ]


dtype('float64')

# Question 2

Explain your code and procedure

In [3]:
# code

# Question 3

Explain your code and procedure

In [4]:
# code

ADD CONCISE CONLUSION.